In [ ]:
import subprocess

import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

In [ ]:
import itertools

# get all AvsC peaks across time

In [ ]:
def merge_all_ac_peaks(wkdir, f_input_peak, f_out, fout2):
    """
    """
    
    cmd0 = ['cat'] + f_input_peak
    cmd1 = ['sort', '-k1,1V', '-k2,2n']
    cmd2 = ['cut', '-f', "1-3,8", f_out]
    cmd3 = ['uniq']

    with open(f_out, 'w') as f:
        run0 = subprocess.run(cmd0, cwd=wkdir, stdout=subprocess.PIPE,)
        run1 = subprocess.run(cmd1, cwd=wkdir, stdout=f, input=run0.stdout)

    with open(f_out2, 'w') as f:
        run2 = subprocess.run(cmd2, cwd=wkdir, stdout=subprocess.PIPE,)
        run3 = subprocess.run(cmd3, cwd=wkdir, stdout=f, input=run2.stdout)
        
    return 

In [ ]:
wkdir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac'
ts = [6,8,10,12,14,17,21]
AvsCs = ['A', 'C']
PvsNs = ['Bp', 'Bn']

In [ ]:
f_input_peak = [f"P{ti}_{PvsN}_peaks_ngene.bed" for ti, PvsN in itertools.product(ts, PvsNs)]
f_out   = wkdir + '/all_BpvsBn_peaks.bed'
f_out2  = wkdir + '/all_BpvsBn_peaks_unique.bed'

merge_all_ac_peaks(wkdir, f_input_peak, f_out, f_out2)

In [ ]:
f_input_peak = [f"P{ti}_Bp_peaks_ngene.bed" for ti in ts]
f_out   = wkdir + '/all_Bp_peaks.bed'
f_out2  = wkdir + '/all_Bp_peaks_unique.bed'

merge_all_ac_peaks(wkdir, f_input_peak, f_out, f_out2)

In [ ]:
f_input_peak = [f"P{ti}_Bn_peaks_ngene.bed" for ti in ts]
f_out   = wkdir + '/all_Bn_peaks.bed'
f_out2  = wkdir + '/all_Bn_peaks_unique.bed'

merge_all_ac_peaks(wkdir, f_input_peak, f_out, f_out2)

In [ ]:
f_input_peak = [f"P{ti}_{AvsC}_peaks_ngene.bed" for ti, AvsC in itertools.product(ts, AvsCs)]
f_out   = wkdir + '/all_AvsC_peaks.bed'
f_out2  = wkdir + '/all_AvsC_peaks_unique.bed'

merge_all_ac_peaks(wkdir, f_input_peak, f_out, f_out2)

In [ ]:
f_input_peak = [f"P{ti}_A_peaks_ngene.bed" for ti in ts]
f_out   = wkdir + '/all_A_peaks.bed'
f_out2  = wkdir + '/all_A_peaks_unique.bed'

merge_all_ac_peaks(wkdir, f_input_peak, f_out, f_out2)

In [ ]:
f_input_peak = [f"P{ti}_C_peaks_ngene.bed" for ti in ts]
f_out   = wkdir + '/all_C_peaks.bed'
f_out2  = wkdir + '/all_C_peaks_unique.bed'

merge_all_ac_peaks(wkdir, f_input_peak, f_out, f_out2)

# get peaks whose cloest genes is Rfx3

In [ ]:
def get_peaks_closest_gene(target_gene, f_in, f_out, wkdir):
    """
    """
    
    command = ['grep', target_gene, f_in]
    with open(f_out, 'w') as f:
        subprocess.run(command, cwd=wkdir, stdout=f, stderr=subprocess.PIPE) # , text=True)
    
    return


In [ ]:
target_gene = 'Rfx3'
f_in  = wkdir + '/all_AvsC_peaks_unique.bed' 
f_out = wkdir + f"/peaks_closest_to_{target_gene}.bed"
get_peaks_closest_gene(target_gene, f_in, f_out, wkdir)

In [ ]:
target_gene = 'Foxp1'
f_in  = wkdir + '/all_AvsC_peaks_unique.bed' 
f_out = wkdir + f"/peaks_closest_to_{target_gene}.bed"
get_peaks_closest_gene(target_gene, f_in, f_out, wkdir)

In [ ]:
target_gene = 'Meis2'
f_in  = wkdir + '/all_AvsC_peaks_unique.bed' 
f_out = wkdir + f"/peaks_closest_to_{target_gene}.bed"
get_peaks_closest_gene(target_gene, f_in, f_out, wkdir)

# get peaks that have specific binding sites 

In [ ]:
def get_peaks_tfbs(tf_motif, f_input_fimo, f_input_peak, fout, wkdir):
    """
    """
    
    # get the binding sites for the specific motif
    cmd0 = ['grep', tf_motif] + f_input_fimo
    # only where they are 
    cmd1 = ['cut', '-f', '3,4,5']
    # sort 
    cmd2 = ['sort', '-k1,1V', '-k2,2n']
    # uniq
    cmd3 = ['uniq']
    # assign them to peaks
    cmd4 = ['bedtools', 'closest', '-d', '-a', '-', '-b', f_input_peak]

    with open(f_out, 'w') as f:
        run0 = subprocess.run(cmd0, cwd=wkdir, input=None,        stdout=subprocess.PIPE)
        run1 = subprocess.run(cmd1, cwd=wkdir, input=run0.stdout, stdout=subprocess.PIPE)
        run2 = subprocess.run(cmd2, cwd=wkdir, input=run1.stdout, stdout=subprocess.PIPE)
        run3 = subprocess.run(cmd3, cwd=wkdir, input=run2.stdout, stdout=subprocess.PIPE)
        run4 = subprocess.run(cmd4, cwd=wkdir, input=run3.stdout, stdout=f)

    return 

In [ ]:
# get Rfx3 binding sites - clean up 

f_input_fimo = [f"fimo_P{ti}_{AvsC}.tsv" for ti, AvsC in itertools.product(ts, AvsCs)]
f_input_peak = 'all_AvsC_peaks_unique.bed'

tf_motif = 'RFX3'
f_out = wkdir + f"/peaks_tfbs_{tf_motif}.bed"
get_peaks_tfbs(tf_motif, f_input_fimo, f_input_peak, f_out, wkdir)

In [ ]:

f_input_fimo = [f"fimo_P{ti}_{AvsC}.tsv" for ti, AvsC in itertools.product(ts, AvsCs)]
f_input_peak = 'all_AvsC_peaks_unique.bed'

tf_motif = 'FOXP1'
f_out = wkdir + f"/peaks_tfbs_{tf_motif}.bed"

get_peaks_tfbs(tf_motif, f_input_fimo, f_input_peak, f_out, wkdir)

In [ ]:

f_input_fimo = [f"fimo_P{ti}_{AvsC}.tsv" for ti, AvsC in itertools.product(ts, AvsCs)]
f_input_peak = 'all_AvsC_peaks_unique.bed'

tf_motif = 'MEIS2'
f_out = wkdir + f"/peaks_tfbs_{tf_motif}.bed"

get_peaks_tfbs(tf_motif, f_input_fimo, f_input_peak, f_out, wkdir)

# analyze these peaks

In [ ]:

fs = [
    wkdir + '/peaks_tfbs_RFX3.bed',
    wkdir + '/peaks_tfbs_FOXP1.bed',
    wkdir + '/peaks_tfbs_MEIS2.bed',
] 

for f in fs:
    df_orig = pd.read_csv(f, header=None, sep='\t')
    
    dfp = df_orig.groupby([3,4,5]).size().sort_values()
    
    dfg = df_orig.groupby(6).size().sort_values()
    
    dfp2g = df_orig.groupby([3,4,5]).first() 
    dfp2g = dfp2g.groupby(6).size().sort_values()
    
    print(f)
    print(dfp.shape, dfp.max())
    print(dfg.shape, dfg.max(), dfp2g.shape, dfp2g.max())

In [ ]:
dfp2g